In [1]:
!pip install sentence_transformers pinecone-client
!pip install neo4j

In [16]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import os, json
import numpy as np

In [1]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import requests, re
import pinecone
import warnings
warnings.filterwarnings("ignore")

C:\Users\zhang\.conda\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import neo4j
from neo4j import GraphDatabase
import re

In [3]:
#triplet retrieval pipeline parameters

url = "neo4j+s://ef25c60e.databases.neo4j.io:7687"
username = "username"
password = "password"

graphDB_Driver = GraphDatabase.driver(url, auth=(username, password))

# rel_str = 'ActTowards, CommunicatesWith, InteractsWith, Supplies, Demands, Acquires, Transforms, Decides, Assesses, Solves, Develops, Impacts, Manages, Moves, Happens'
# node_str = 'Person, Company, Organization, Facility, Location, GeoPoliticalEntity, Time, Date, Event, Product, Regulation'

rel_str = 'ActTowards, CommunicatesWith, InteractsWith, Supplies, Demands, Acquires, Transforms, Decides, Assesses, Solves, Develops, Impacts, Manages, Moves, Happens, Produce, is'
node_str = 'Person, Company, Organization, Facility, Location, Country, GeoPoliticalEntity, Time, Date, Event, Product, Regulation, Journal, Paper, Number, Problem'

In [4]:
import re
#triplet retrieval
def find_match_query(input_string):
    pattern1 = re.compile(r'MATCH.*?\n', re.DOTALL)
    match1 = pattern1.search(input_string)
    pattern2 = re.compile(r'MATCH.*?\.', re.DOTALL)
    match2 = pattern2.search(input_string)

    if match1 and not match2:
        return match1.group().rstrip().rstrip('.').rstrip(',')
    elif match2 and not match1:
        return match2.group().rstrip().rstrip('.').rstrip(',')
    elif not match1 and not match2:
        return None
    elif len(match1.group()) < len(match2.group()):
        return match1.group().rstrip().rstrip('.').rstrip(',')
    else:
        return match2.group().rstrip().rstrip('.').rstrip(',')


# def find_match_query_ver2(input_string):
#     pattern = re.compile(r'MATCH.*?[.\n]', re.DOTALL)
#     matches = pattern.findall(input_string)

#     if not matches:
#         return None

#     chosen_match = min(matches, key=len)
#     return chosen_match.rstrip().rstrip('.').rstrip(',')


def output_triples(output_query, record_names):
    pattern = r'\((\w+):(\w+)\)-\[:(\w+)\]->\((\w+):(\w+)\)\s*RETURN (\w+)'
    match = re.search(pattern, output_query)
    if match:
        s, stype, p, o, otype, var = match.groups()
    else:
        return None

    if var == s:
        return [(n, p, o) for n in record_names]
    elif var == o:
        return [(s, p, n) for n in record_names]
    elif var == p:
        return [(s, n, o) for n in record_names]
    else:
        return None

In [5]:
def retrieve_triples(question, model, node=node_str, rel=rel_str, api_token= "apitoken", max_new_token=100):


#     prompt = f'''Cypher is a query language designed for querying graph databases. It was initially developed by Neo4j, and is now an open standard for various graph databases.

# There are pre-defined node labels in our Neo4j database, namely {node}. There are also 15 pre-defined relationship types, namely {rel}.

# Here's an example of a Cypher query for a natural language query "What companies are the suppliers of Louis Vuitton?":

# MATCH (company:Company)-[:Supplies]->(Louis_Vuitton:Company) RETURN company

# This query retrieves companies from the class "Company" that are connected with Intel by a "Supplies" relationship, and then returns those companies. Never forget to return values in a query.

# Using the node labels and the relationship types in the database, for the query in natural language "{question}," the corresponding Cypher query should be '''


    prompt = f'''Cypher is a query language designed for querying graph databases. It was initially developed by Neo4j, and is now an open standard for various graph databases.

There are pre-defined node labels in our Neo4j database, namely {node}. There are also 15 pre-defined relationship types, namely {rel}.

Here's an example of a Cypher query for a natural language query "What companies are the suppliers of Louis Vuitton?":
MATCH (company:Company)-[:Supplies]->(Louis_Vuitton:Company) RETURN company
This query retrieves companies from the class "Company" that are connected with Intel by a "Supplies" relationship, and then returns those companies.

Here's another example for query "Who does China trade with?":
MATCH (China:Country)-[:InteractsWith]->(other_country:Country) RETURN other_country
This query retrieves countries from the class "Country" that are connected with China by a "InteractsWith" relationship, as the predicate "trade with" closely aligns with the pre-defined relationship type "InteractsWith".

Never forget to return values in a query. Using the node labels and the relationship types in the database, for the query in natural language "{question}," the corresponding Cypher query should be '''

    client = InferenceClient(model=model, token=api_token)
    output_text = client.text_generation(prompt, max_new_tokens=max_new_token)
    try:
        output_query = find_match_query(output_text) + '.name'
    except:
        output_query = ""
    try:
        db_records, summary, keys = graphDB_Driver.execute_query(output_query, database_="neo4j")
        record_names = [r[keys[0]] for r in db_records]
    except:
        record_names = []
    output = output_triples(output_query, record_names)
    output_str = []
    if output is not None:
        for triplet in output:
            if type(triplet[0])==str and type(triplet[1])==str and type(triplet[2])==str: #correct output type
                output_str.append(triplet)



    return output_str

In [6]:
def read_file_and_process_sentences(file_path):
    sentences = []
    with open(file_path, 'r',encoding="utf-8") as file:
        for line in file:

            cleaned_line = line.strip()
            if cleaned_line:  # Check if the line is not empty after stripping
                sentences.append(cleaned_line)
    return sentences


In [7]:
def sentence_embeddings(sentences, model='sentence-transformers/all-MiniLM-L6-v2'):
    # generate embedding vectors for sentences.
    # sentences: list of input sentences,
    # model: sentence-transformers/all-MiniLM-L6-v2 as default
    model = SentenceTransformer(model)
    embeddings = model.encode(sentences,batch_size=128)
    return embeddings

In [8]:
def similarity_search(sentence_vectors, top_k=5, api_key="apikey", environment="gcp-starter", index="textsearch"):
    # return the top_k similar sentences for ench sentence_vector in inputs, inputs shape (n, )
    # output list of shape (n, top_k)
    pinecone.init(api_key=api_key,environment=environment)
    index = pinecone.Index("textsearch")
    # print(index.describe_index_stats())

    output_sentences=[]
    for vector in sentence_vectors:
        results = index.query(vector= vector.tolist(), top_k=top_k,include_metadata=True)
        sentence=[]
        for match_dict in results['matches']:
            sentence.append(match_dict['metadata']['text'][1:-1]+".".strip())
        output_sentences.append(sentence)
    return output_sentences

In [9]:
# Some Back-up Prompts

# prompt = f'''
# RDF stands for Resource Description Framework, and it is a widely used format for representing information in the form of subject-predicate-object triplets in Knowledge Graph. Each RDF triplet consists of three components:

# Subject: The subject is typically a resource or entity that you want to describe.
# Predicate: The predicate, also known as the property or relationship, describes the relationship between the subject and the object.
# Object: The object is the value or target of the statement.

# Suppose you are an RDF triplet extraction model. I will give you 15 categories of predicates in the field of supply chain.

# {category}

# Give you some sentences, please extract all the RDF triplets based on the categories. Please form each triplet as (subject, predicate, object) + (predicate category) in the output. If no triplet is found, answer: None. Do not include other words in the answer.

# Sentence: {sentence}

# Triplets: '''


# Prompt with no predicate types

# prompt = f'''
# In Knowledge Graphs, Resource Description Framework (RDF) serves as a widely adopted format for expressing information using subject-predicate-object triplets. Each RDF triplet consists of three primary components. The subject usually represents an entity. The predicate defines the connection between the subject and the object. The object is the value or target of the statement.

# For instance, consider the sentence "Apple contributes to the economy of China." In this case, the subject, predicate, and object are respectively "Apple," "contribute to," and "the economy of China." We format the triplet output as (subject, predicate, object), which is (Apple, contribute to, the economy of China).

# It's important to note that a single sentence can contain multiple triplets. For example, in the sentence, "Walmart was trial-testing a service it developed with IBM to monitor produce," the output should consist of several triplets, including (Walmart, test, a service), (Walmart, develop with, IBM), and (Walmart, monitor, produce).

# Given the above information, for the sentence "{sentence}," the correct output should be '''

In [10]:
def triplet_extraction(sentences, start_idx, end_idx, category, model, api_token="apitoken", max_new_token=100):

    # extract triples of given setences, using giving model:"meta-llama/Llama-2-70b-chat-hf","meta-llama/Llama-2-70b-chat-hf", "bigscience/bloom-560m","bigscience/bloom-3b"
    # input shape (n,1), output an excel with column：sentence_index	subject	predicate	object	predicate_type

    client = InferenceClient(model=model, token=api_token)

    pattern = r'\(([^,]+), ([^,]+), ([^)]+)\) \+ \(([^)]+)\)'
    # pattern = r'\(([^,]+), ([^,]+), ([^)]+)\)'

    df = pd.DataFrame(columns = ['sentence_index', 'subject', 'predicate', 'object', 'predicate_type'])
    # df = pd.DataFrame(columns = ['sentence_index', 'subject', 'predicate', 'object'])

    sentence_idx = start_idx

    for sentence in sentences[start_idx:end_idx]:

        # Use a few-shot prompt
        prompt = f'''
In Knowledge Graphs, Resource Description Framework (RDF) serves as a widely adopted format for expressing information using subject-predicate-object triplets. Each RDF triplet consists of three primary components. The subject usually represents an entity. The predicate defines the connection between the subject and the object. The object is the value or target of the statement.

In the field of supply chain, there are 15 pre-defined categories of predicates, which are as follows.

{category}

For instance, consider the sentence "Apple contributes to the economy of China." In this case, the subject, predicate, and object are respectively "Apple," "contribute to," and "the economy of China." The predicate "contribute to" belongs to the category of "Impacts." Consequently, we format the triplet output as (subject, predicate, object) + (predicate category), which is (Apple, contribute to, the economy of China) + (Impacts) for this sentence.

It's important to note that a single sentence can contain multiple triplets. For example, in the sentence, "Walmart was trial-testing a service it developed with IBM to monitor produce," the output should consist of several triplets, including (Walmart, trial-test, a service) + (Assesses), (Walmart, develop with, IBM) + (Develops), (Walmart, monitor, production) + (Assesses).

Given the above information, for the sentence "{sentence}," the correct output should be '''

        output_text = client.text_generation(prompt, max_new_tokens=max_new_token)
        # print(sentence)
        # print(output_text)

        matches = re.findall(pattern, output_text)
        for m in matches:
            phrase1, phrase2, phrase3, phrase4 = m
            new_row = {'sentence_index': sentence_idx, 'subject': phrase1, 'predicate': phrase2, 'object': phrase3, 'predicate_type': phrase4}
            df = df.append(new_row, ignore_index=True)

        sentence_idx += 1

    return df
    # df.to_csv(output_file, index=False)

In [11]:
def chatbot_pipeline(questions, start_idx, end_idx, model, category, max_new_token=300, api_token="apitoken"):

    # generate answers for questions
    # store triplets for questions and triplets,output answers in answers.txt
    model_name=model.split("/")[-1]
    client = InferenceClient(model=model, token=api_token)
    # triplet retrival
    #If there are relevant triplets,keep the triplets,else keep the question for sentence similarity search

    print("Triplet Retrieval")

    sentences_list=[]

    for i in tqdm(range(start_idx, end_idx)):
        triplet_list = retrieve_triples(question=questions[i], model=model)
        
        print(i,triplet_list)

        if triplet_list is not None and len(triplet_list)!=0:
            triplet_list_sentence = [" ".join(triplet_list[i])+"." for i in range(0,len(triplet_list))] #list of sentences
            sentences_list.append(triplet_list_sentence)
        else:
            sentences_list.append([questions[i]]) #None relevant triplets,keep the questions
   # print("sentence_list:",sentences_list)

    print("Sentence Embedding and Similarity search")
    sentence_vectors=[]
    background_sentences=[]
    for i in tqdm(range(0,len(sentences_list))):
        #print(sentences_list[i])
        sentence_vector = sentence_embeddings(sentences_list[i])
        #print(len(sentence_vector))
        sentence_vectors.append(sentence_vector)
        if len(sentence_vector)==1:

            background_sentence = list(set(similarity_search(sentence_vector,top_k=5)[0])) #more sentences for questions

        else:
            background_sentence = list(set([sentence[0] for sentence in similarity_search(sentence_vector,top_k=1)]))
            if len(background_sentence)>=20:
                #chunk background sentence
                background_sentence=background_sentence[0:20] 
                
        background_sentences.append(background_sentence)
        #print("back:",background_sentences)





    #print("back:",background_sentences)
    # join background sentences
    background_sentences_join=["" for i in range(0,len(background_sentences))]
    answers=[]
    for i in range(0, len(background_sentences)):
          background_sentences_join[i]= ("\n").join(background_sentences[i]) #join the relevant background information
    #print(background_sentences)
    
    id=start_idx
    with open(model_name+"_"+"background_sentence.txt", "a",encoding='utf-8') as file:
        for sentence in background_sentences_join:
            file.write(str(id)+"<SEP>"+'\n')
            file.write(sentence + '\n')
            id=id+1

    print("Extracting triple from questions")
    df_questions = pd.DataFrame(columns = ['question_index','sentence_index', 'subject', 'predicate', 'object', 'predicate_type'])#might overflow
    # df_questions = pd.DataFrame(columns = ['question_index','sentence_index', 'subject', 'predicate', 'object'])#might overflow
    for i in tqdm(range(start_idx, end_idx)):
        df = triplet_extraction(background_sentences[i-start_idx],start_idx=0, end_idx=len(background_sentences[i-start_idx]), category=category,model=model)
        df['question_index'] = [i for j in range(0,len(df))]
        df = df[['question_index', 'sentence_index', 'subject', 'predicate', 'object', 'predicate_type']]
        # df = df[['question_index', 'sentence_index', 'subject', 'predicate', 'object']]
        df_questions = pd.concat([df_questions,df], ignore_index=True)
    df_questions = df_questions.drop('sentence_index', axis=1)
    df_questions.to_csv(model_name+"_"+f'''questions_triples_{start_idx}_{end_idx}.csv''', index=False)

    print("Generating answers")
    for i in tqdm(range(start_idx,end_idx)):
        prompt = f'''
        I will give you a question and related information. Please answer the question based on the given related information. Do not include other information.
        Question: {questions[i]}
        Related information:{background_sentences_join[i-start_idx]}

        Answer:'''
        output_text = client.text_generation(prompt,max_new_tokens=max_new_token)
        # print(prompt)
        # print(output_text)
        answers.append(output_text.strip())
    id=start_idx
    with open(model_name+"_"+"answers.txt", "a",encoding='utf-8') as file:
        for answer in answers:
            file.write(str(id)+"<SEP>" '\n')
            file.write(answer + '\n')
            id=id+1

    print("Extracting triple from answers")
    answer_sentences=[answers[i].split(".") for i in range(0,len(answers))]
    df_answers = pd.DataFrame(columns = ['answer_index','sentence_index', 'subject', 'predicate', 'object', 'predicate_type']) # might overflow
    # df_answers = pd.DataFrame(columns = ['answer_index','sentence_index', 'subject', 'predicate', 'object']) # might overflow
    for i in tqdm(range(start_idx,end_idx)):

        df=triplet_extraction(answer_sentences[i-start_idx],start_idx=0,end_idx=len(answer_sentences[i-start_idx]), category=category,model=model)
        df['answer_index'] = [i for j in range(0,len(df))]
        df = df[['answer_index','sentence_index', 'subject', 'predicate', 'object', 'predicate_type']]
        # df = df[['answer_index','sentence_index', 'subject', 'predicate', 'object']]
        df_answers=pd.concat([df_answers,df], ignore_index=True)
    df_answers=df_answers.drop('sentence_index', axis=1)

    df_answers.to_csv(model_name+"_"+f'''answers_triples_{start_idx}_{end_idx}.csv''', index=False)

In [12]:
category = '''Category 1: ActTowards
Produce, create, present, implement, construct, operate, perform, conduct, run, tend, carry, start, build

Category 2: CommunicatesWith
describe, express, indicate, note, suggest, propose, discuss, explain, report, represent, illustrate, refer, mention, reveal, specify

Category 3: InteractsWith
connect, relate, associate, link, integrate, combine, incorporate, meet, interact, engage, cooperate, communicate, collaborate

Category 4: Supplies
give, share, offer, provide, supply, deliver

Category 5: Demands
need, require, demand, depend

Category 6: Acquires
order, obtain, purchase, buy, acquire, receive, trade, sell

Category 7: Transforms
change, transform, adapt, shift

Category 8: Decides
choose, decide, select, pick, opt

Category 9: Assesses
measure, assess, evaluate, calculate, test, examine, investigate

Category 10: Solves
solve, fix, resolve, address

Category 11: Develops
increase, develop, improve, grow, progress, advance, evolve, proceed

Category 12: Impacts
affect, influence, Impact, contribute, facilitate

Category 13: Manages
manage, control, coordinate, regulate, arrange, organize, sort

Category 14: Moves
move, go, travel, journey, walk

Category 15: Happens
occur, happen, continue, remain'''

In [13]:
def remove_parentheses(texts):
    # Using regular expression to remove content within parentheses including the parentheses themselves
    return [re.sub(r'\([^)]*\)', '', text) for text in texts]


In [38]:
questions=read_file_and_process_sentences("questions_final_100.txt")

print(questions[0:10])

['What are the properties of mulit product supply chain?', 'What are supply chain challenges when running a food business?', 'What is green supply chain management?', 'What are some driving factors of green supply chain management in New York?', 'What is Deep Game Framework for supply chain analysis?', 'Can you give an example of a sustainable business model?', 'How does pandemic risk management differ from regular times in terms of supply chain?', 'What impacts did COVIS-19 have on global supply chain?', 'How could supply chain issues been avoided caused from COVID-19?', 'How has supply chain played a role in post COVID inflation']


In [16]:
questions=read_file_and_process_sentences("questions_updated.txt")
questions=remove_parentheses(questions)
print(questions[25:50])

['How much is the world population increasing by by 2050? ', 'The Dow Chemical Company manufactures how many product families? ', 'The Dow Chemical Company manufactures\tat how many sites? ', 'The Dow Chemical Company manufactures in how many countries? ', 'What company serves 45000 customer locations daily? ', 'What was the $787 billion US Congress Bill passed in 2009? ', 'Spatial concentration of demand is measured by what? ', 'Green hydrogen can help decarbonize parts of what sector? ', 'What year did the Renewable Energy Sources Act begin? ', 'What government enacted the Renewable Energy Sources Act? ', 'What is one reason supply chain management seems to be struggling? ', 'What is the backbone of the global economy? ', 'What is a market relevant to supply chain where prices are determined by a spot market? ', 'US corn, soybeans, wheat and tobacco industries are what type of market? ', 'The war in which country has disrupted supply chains? ', 'The United States have been in trade d

In [20]:
model = "meta-llama/Llama-2-7b-chat-hf"

chatbot_pipeline(questions, start_idx=25, end_idx=len(questions), model=model, category=category, max_new_token=100)

Triplet Retrieval


  4%|███▎                                                                               | 1/25 [00:03<01:35,  3.98s/it]

25 []


  8%|██████▋                                                                            | 2/25 [00:07<01:20,  3.49s/it]

26 []


...

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [04:28<00:00, 10.72s/it]


Generating answers


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:16<00:00,  1.55it/s]


Extracting triple from answers


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:40<00:00,  1.60s/it]


In [22]:
 print(retrieve_triples("'What companies are the developers of Intel?'", model))

[('Walmart', 'Develops', 'Intel'), ('Company', 'Develops', 'Intel'), ('VCF', 'Develops', 'Intel'), ('suppliers', 'Develops', 'Intel'), ('Wilhelm', 'Develops', 'Intel'), ('Apple', 'Develops', 'Intel'), ('Apple', 'Develops', 'Intel'), ('SCN', 'Develops', 'Intel'), ('SCN', 'Develops', 'Intel'), ('IBM', 'Develops', 'Intel'), ('A', 'Develops', 'Intel'), ('Publ', 'Develops', 'Intel'), ('Amazon', 'Develops', 'Intel'), ('Company', 'Develops', 'Intel'), ('Company', 'Develops', 'Intel'), ('Company', 'Develops', 'Intel'), ('CSR', 'Develops', 'Intel'), ('supply_chain_management', 'Develops', 'Intel'), ('Ferrari', 'Develops', 'Intel'), ('Company', 'Develops', 'Intel'), ('SCRES', 'Develops', 'Intel'), ('Walmart', 'Develops', 'Intel'), ('Company', 'Develops', 'Intel'), ('Walmart', 'Develops', 'Intel'), ('developing', 'Develops', 'Intel'), ('Walmart', 'Develops', 'Intel'), ('Company', 'Develops', 'Intel'), ('CLSCs', 'Develops', 'Intel'), ('Maersk', 'Develops', 'Intel'), ('Walmart', 'Develops', 'Intel'